In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import httpx
import lxml
import re

from lxml import html

In [3]:
headers = {}
cookies = {'age_check': '1'}

In [4]:
genre_list = httpx.get('https://blog.fc2.com/genrelist/', headers=headers, cookies=cookies)
genre_groups = html.fromstring(genre_list.text).xpath('//*[@class="genre_group"]')
data = []
for group in genre_groups:
    genre = group.xpath('./h3/a')[0]
    genre_id = genre.attrib['href'].strip().split('/')[-2]
    genre_text = genre.text.strip()
    
    children = []
    subgenres = group.xpath('./ul/li/a')
    for node in subgenres:
        subgenre_id = re.findall(r'/subgenre/([0-9]+)/', node.attrib['href'])[0]
        subgenre_text = ''.join(node.itertext()).strip()
        children.append({
            'id': subgenre_id,
            'name': subgenre_text
        })

    data.append({
        'id': genre_id,
        'name': genre_text,
        'subgenre': children
    })

data[:3], data[-3:]

([{'id': '7',
   'name': '日記',
   'subgenre': [{'id': '1', 'name': '小学生'},
    {'id': '2', 'name': '中学生'},
    {'id': '3', 'name': '高校生'},
    {'id': '4', 'name': '専門学校生'},
    {'id': '5', 'name': '大学生'},
    {'id': '6', 'name': '会社員・OL'},
    {'id': '7', 'name': '主婦'},
    {'id': '8', 'name': '派遣社員'},
    {'id': '352', 'name': 'ゆとり世代'},
    {'id': '354', 'name': '家族日記'},
    {'id': '353', 'name': '創作日記'},
    {'id': '9', 'name': 'その他'}]},
  {'id': '6',
   'name': 'ニュース',
   'subgenre': [{'id': '355', 'name': '時事'},
    {'id': '10', 'name': '芸能'},
    {'id': '11', 'name': 'スポーツ'},
    {'id': '12', 'name': 'IT・インターネット'},
    {'id': '13', 'name': 'ガジェット'},
    {'id': '14', 'name': 'ローカル'},
    {'id': '15', 'name': '海外'},
    {'id': '16', 'name': 'その他'}]},
  {'id': '22',
   'name': 'ブログ',
   'subgenre': [{'id': '17', 'name': '絵日記'},
    {'id': '18', 'name': 'イラストブログ'},
    {'id': '19', 'name': '写真ブログ'},
    {'id': '20', 'name': '社長ブログ'},
    {'id': '358', 'name': '店長ブログ'},
    {'id': '359

In [5]:
genre_list = httpx.get('https://blog.fc2.com/genrelist/', headers=headers, cookies=cookies)
genre_anchor_nodes = html.fromstring(genre_list.text).xpath('//*[@class="subgenre_name_list"]/li/a')
genre_urls = [(''.join(node.itertext()).strip(), node.attrib['href']) for node in genre_anchor_nodes]
genre_urls[:3], genre_urls[-3:]

([('小学生', '/subgenre/1/'), ('中学生', '/subgenre/2/'), ('高校生', '/subgenre/3/')],
 [('SM', '/a/subgenre/379/'),
  ('ライブチャット', '/a/subgenre/344/'),
  ('その他', '/a/subgenre/345/')])

In [6]:
genre_top = httpx.get('https://blog.fc2.com/a/subgenre/333/', headers=headers, cookies=cookies)
entry_urls = list(set(re.findall(r'https?://.+fc2.com/blog-entry-.+\.html', genre_top.text)))
entry_urls

['http://45kekkon.blog.fc2.com/blog-entry-2052.html',
 'http://hitodumafuuzoku.blog.fc2.com/blog-entry-15925.html',
 'http://etigoya955.blog.fc2.com/blog-entry-3494.html',
 'http://nonkeurisen.blog.fc2.com/blog-entry-903.html',
 'http://kanosanpo.blog96.fc2.com/blog-entry-1560.html',
 'http://avohayo.blog.fc2.com/blog-entry-2197.html',
 'http://ckln.blog.fc2.com/blog-entry-1007.html',
 'http://avohayo.blog.fc2.com/blog-entry-2198.html',
 'http://jkmahiro.blog.fc2.com/blog-entry-2818.html',
 'http://oran.blog2.fc2.com/blog-entry-5011.html',
 'http://avohayo.blog.fc2.com/blog-entry-2184.html',
 'http://avohayo.blog.fc2.com/blog-entry-2202.html',
 'http://kanosanpo.blog96.fc2.com/blog-entry-1570.html',
 'http://avohayo.blog.fc2.com/blog-entry-2200.html',
 'http://gurutotoki.blog.fc2.com/blog-entry-12306.html',
 'http://sumishisumi.blog.fc2.com/blog-entry-241.html',
 'http://oran.blog2.fc2.com/blog-entry-5013.html',
 'http://frenum2.blog.fc2.com/blog-entry-1015.html',
 'http://oran.blog2.f

In [7]:
pager_anchor_nodes = html.fromstring(genre_top.text).xpath('//*[@class="pager"]/a')
next_page = [node.attrib['href'] for node in pager_anchor_nodes if node.text == '>']

In [8]:
def extractcontent(html):
    from extractcontent3 import ExtractContent
    extractor = ExtractContent()

    opt = {"threshold":50}
    extractor.set_option(opt)

    extractor.analyse(html)
    text, title = extractor.as_text()
    html, title = extractor.as_html()
    title = extractor.extract_title(html)
    return text.strip()

def readability_html2text(html):
    import html2text
    from readability.readability import Document
    article = Document(html).summary()
    text = html2text.html2text(article)
    return text.strip()

def readability_lxml(html):
    import html2text
    from readability.readability import Document
    article = Document(html).summary()
    text = lxml.html.fromstring(article).text_content().strip()
    return text

def tura(html):
    from trafilatura import fetch_url, extract
    text = extract(html)
    return text

def test(url):
    response = httpx.get(url)
    html = response.text
    print('[ExtractContent3]\n' + extractcontent(html))
    print('\n' + '-'*100 + '\n')
    print('[Readability (html2text)]\n' + readability_html2text(html))
    print('\n' + '-'*100 + '\n')
    print('[Readability (lxml)]\n' + readability_lxml(html))
    print('\n' + '-'*100 + '\n')
    print('[Trafilatura]\n' + tura(html))

In [9]:
test('http://kobedancestudioway.blog91.fc2.com/blog-entry-2484.html')

[ExtractContent3]
11
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 01
Author:神戸　板宿ダンススクール･スタジオWAY
ホームページ：http://www.p-kit.com/hp/studioway/神戸市須磨区板宿駅上すぐ30秒!!ダンススタジオの「スタジオＷＡＹ」(スタジオウエイ)ですAスタジオ 社交教室Bスタジオ ダンススクール キッズバレエ、ミュージカル、ヒップホップ、ジャズ、ガールズヒップホップ、チアダンスとさまざまなジャンルのレッスンがございます（＾＾）

----------------------------------------------------------------------------------------------------

[Readability (html2text)]
今日はママヒップホップの日です

このクラスは皆さん大体同じ年代なので なんの気兼ねもなく入れるから良いよねぇ

かるーい気持ちで入れるぅ

1週間の内で一番楽しい時間なんですぅ

という話し声が聞こえてきました

ほんっ とうに よかった このクラスを作って

って感じになりますよね

そんなクラスの先生は

今日、パパの誕生日なのでプレゼントを送ったんですよー 結構良いものを！

なのに ありがとう の連絡もないんですよー

有難うは？ の連絡をした方が良いのかなぁ？

あれ？ おめでとうの連絡をした方が良いのかな？

という とっても親孝行な先生なのです

雰囲気の良いクラスになのできっと入りやすいと思いますよぉ

[![](https://blog-
imgs-149.fc2.com/k/o/b/kobedancestudioway/fc2blog_20210619180816630s.jpg)](https://blog-
imgs-149.fc2.com/k/o/b/kobedancestudioway/fc2blog_20210619180816630.jpg)[![](https://blog-
imgs-149.fc2.com/k/o/b/kobedancestudioway/fc

In [10]:
test('http://setakitasofttennis.blog.fc2.com/blog-entry-351.html')

[ExtractContent3]


----------------------------------------------------------------------------------------------------

[Readability (html2text)]
6月29日(火)に卒業アルバムの写真撮影をします。3年生はユニフォームの準備をしておいてください。なお、天候などで延期の可能性があります。

----------------------------------------------------------------------------------------------------

[Readability (lxml)]
6月29日(火)に卒業アルバムの写真撮影をします。3年生はユニフォームの準備をしておいてください。なお、天候などで延期の可能性があります。

----------------------------------------------------------------------------------------------------

[Trafilatura]
瀬田北ソフトテニス
記事一覧
卒業アルバムの写真撮影予定
2021/06/21
01:16
6月29日(火)に卒業アルバムの写真撮影をします。3年生はユニフォームの準備をしておいてください。なお、天候などで延期の可能性があります。
3年生団体研修大会について
ホーム
6月12日（土）
コメント
コメントの投稿
名前
タイトル
メールアドレス
URL
本文
パスワード
非公開コメント
管理者にだけ表示を許可する
プロフィール
Author:setakitasofttennis
ＦＣ２ブログへようこそ！
最新記事
休止のお知らせ (04/01)
3月25日（土） (03/25)
3月23日 (03/22)
3月28日29日の試合について (03/18)
春休み後半の予定 (03/18)
最新コメント
阪口智祐:1月予定表 (12/22)
:11月26日（土）について (11/19)
setakitasofttennis:10月29日の練習試合 (10/26)
setakitasofttennis:11月予定表 (10/25)
:10

In [11]:
test('https://pool330.blog.fc2.com/blog-entry-3530.html')

[ExtractContent3]
競輪場通り、ユニクロ宇都宮戸祭店隣にある「とんかつ　あじでん」。この日も店前の駐車場は空きはなし。いつもはここで諦めてしまうのですが、どうしても「とんかつ」が食べたいモード。お店の裏側にも駐車場があり、ぐるり回って様子を見に行くと・・・。（裏側は共同駐車場になっており、停められる場所には看板が立ってます）。空きスペース発見、車を停めお店脇の通路を通り玄関口へ。新型コロナの感染対策で、一席飛ばしのカウンター席へ。厨房からとんかつを揚げる音が聞こえてくる。今日はグランドメニューから、「上ロース」や「ジャンボロース」、「ねぎおろしロース」など多彩なとんかつを頂こうと思ったのですが、リーズナブルな「サービスランチ」にグラリ・・・。（土日も注文OK）この日も結局「サービスランチ」の中から「とんロースかつランチ」１１００円（税込）を選んでました。ひじきの小鉢と香の物、そしてすりゴマが運ばれ、準備開始。山盛りのキャベツに揚げたてロースかつ。特製のドレッシングをキャベツにかけ、まずはベジファースト。丼にたっぷりご飯と、とん汁も付いてきます。ほんのりピンクがかった綺麗なお肉。柔らかく、サイズも質も満足のとんかつでした。１１００円のサービスランチがこの内容なら、上ロース定食やジャンボロース定食はどんな感じなんだろう～。次回こそは・・と思いながら、お店を後にしました。「とんかつ　あじでん」住所：宇都宮市下戸祭2-20-6電話番号：028-625-2939定休日：水曜日営業時間：１１時～１４時、１７時～２１時栃ナビＨＰ

----------------------------------------------------------------------------------------------------

[Readability (html2text)]
競輪場通り、ユニクロ宇都宮戸祭店隣にある「とんかつ あじでん」。

この日も店前の駐車場は空きはなし。

いつもはここで諦めてしまうのですが、どうしても「とんかつ」が食べたいモード。

お店の裏側にも駐車場があり、ぐるり回って様子を見に行くと・・・。（裏側は共同駐車場になっており、停められる場所には看板が立ってます）。空きスペース発見、車を停めお店脇の通路を通り玄関口へ。

[![とん

In [12]:
test('https://onlinegamedeai.fc2.xxx/blog-entry-1.html')

[ExtractContent3]
近年、様々な技術が発達してインターネットでのサービスが充実してきました。それによって私達の生活も便利になり、今や欠かせないモノとなってきています。デリバリー・銀行・通販・会社とのビデオ通話など、今後も主流となっていくサービスなのは間違いないでしょう。さて、２０２０年（令和２年）の３月末から、世界的にパンデミックが起こりました。新型コロナウイルスです。非常に強い感染力を持ち、人との接触により感染していく未知の病原体です。これによって、人との過度な接触を避けるために不要・不急の外出をしない自粛を世界各国で求められています。未だ経験したことのない脅威にさらされ、人類は緊急事態宣言を行いました。経済に大打撃なのはもちろんですけど、一番怖いのは脅威のウイルスが人命に関わるということでしょう。感染しない・人に感染させないのを基本とし、この未曾有の危機を乗り越えなくてはなりません！でも、人類の発展と文化を止めるわけにはいかないし、私達も生活していかなくてはなりません。当サイトは、オンラインゲームで知り合う方法を載せています！ゲームを探すなら 恋愛ゲーム白書 名作携帯ゲーム リアルな出会い探し穴場の出会いアプリ 近所で恋人探し マッチングアプリ婚活

----------------------------------------------------------------------------------------------------

[Readability (html2text)]
近年、様々な技術が発達して

**インターネット**

でのサービスが充実してきました。

それによって私達の生活も便利になり、今や欠かせないモノとなってきています。

デリバリー・銀行・通販・会社とのビデオ通話など、今後も主流となっていくサービスなのは間違いないでしょう。

さて、２０２０年（令和２年）の３月末から、世界的にパンデミックが起こりました。

 _**新型コロナウイルス**_

です。

非常に強い感染力を持ち、人との接触により感染していく未知の病原体です。

これによって、人との過度な接触を避けるために不要・不急の外出をしない自粛を世界各国で求められています。

未だ経験したことのない脅威にさらされ、人類は緊急事態宣言を行いました。

In [13]:
test("https://tsuchinoko2006.blog.fc2.com/blog-entry-3178.html")

[ExtractContent3]
満月に近いシルエットが煌々と、八鬼山に向かう古道を照らしている。旅に出て5日目の早朝5時30分、しーんと寝静まり返った尾鷲の町を歩き出した。犬の散歩で出会った人以外には誰にも会わずに町を抜け、JR紀勢本線と矢ノ川を渡った。街灯の間隔は歩くほどに遠くなり、ヘッドランプは点けっぱなしだ。トイレと駐車場がある八鬼山道登り口に6時10分到着。一年で一番日が短い時期なので、行動時間が短くなるのはしかたがない。夏ならばもっと早くに出発し、とっくに八鬼山の峠に着いている時間である。駐車場にはヘッドランプを灯し、腰を下ろしている人がいた。これから山を登るハイカーなのか、何者なのかは不明だ。朝の挨拶を一言交わしただけで、通過する。登り口でヘッドランプに照らした道標を確認し、導かれるまま一歩を踏み出す。熊に注意のポスターは慣れっこになってしまったが、不運にも出遭うとすればここからが一番やばそうだなぁ…などと思いながら、真っ暗な石畳を登る。籠立場と町石の案内板に出たところで、ようやく夜が明け始めた。この先には伊勢路最大の難所といわれた七曲の登りとなる。案内板には八鬼山山頂まで2090メートル80分、道標は20/63と刻まれていた。蛇のようにくねる急登の石畳を快調に登っていく。400メートルを一気に登り、24/63の道標を見たところで、七曲は終わったようだった。そこから桜茶屋の一里塚を過ぎ、蓮華石と烏帽子石という巨石が鎮座する石畳を登ると、冷たい風が吹き抜ける九鬼峠に出た。7時30分に荒神堂に到着。ベンチやトイレも整備されており、出発してから初めての休憩を取った。そこから熊注意の看板をチェックしながら林の中を歩くと、標高627メートルの八鬼山峠はいつのまにか過ぎてしまい、芝生に囲まれた「さくらの森広場」に出た。東屋もあって、何といっても朝日に輝く熊野灘の絶景が素晴らしかった。行動食のビスケットを食べたり、カミさんにLINEを送ったりして、しばらく過ごす。誰もいない絶景を独り占めしている至福感は、言葉に言い表せない。これこそが単独行の醍醐味といってもいい。ここから先の下りも同様で、これでもか!と続く石畳を、それこそ鼻歌気分でリズミカルに下りることができた。三木里登り口の63/63の終点の道標に8時50分着。ホテルを出てから3時間20分で、八鬼山を登って下り